In [7]:
from keras.models import load_model
from rbm.core.dice import dice_coef, dice_coef_loss
from rbm.core.utils import min_max_normalization, resample_img
from rbm.core.eval import out_LabelHot_map_2D
from rbm.core.paras import PreParas, KerasParas
from rbm.scripts.rbm import brain_seg_prediction
from pathlib import Path
import SimpleITK as sitk
import numpy as np
from matplotlib import pyplot as plt

In [8]:
# Default Parameters Preparation
pre_paras = PreParas()
pre_paras.patch_dims = [1, 128, 128]
pre_paras.patch_label_dims = [1, 128, 128]
pre_paras.patch_strides = [1, 32, 32]
pre_paras.n_class = 2

# Parameters for Keras model
keras_paras = KerasParas()
keras_paras.outID = 0
keras_paras.thd = 0.5
keras_paras.loss = 'dice_coef_loss'
keras_paras.img_format = 'channels_last'
keras_paras.model_path = Path(r'C:\Users\yoni.browning\OneDrive - Allen Institute\Documents\GitHub\RodentMRISkullStripping\rbm\scripts')/Path( 'rat_brain-2d_unet.hdf5')

In [9]:
model = load_model(keras_paras.model_path,
                         custom_objects={'dice_coef_loss': dice_coef_loss,
                                         'dice_coef': dice_coef})


In [16]:


patch_dims = pre_paras.patch_dims
label_dims = pre_paras.patch_label_dims
strides = pre_paras.patch_strides
n_class = pre_paras.n_class



In [17]:
patches = []
labels = []

image_list = [r'D:\MRI\Nifti\636969\636969_100_raw.nii',
              r'D:\MRI\Nifti\632298\632298_100_raw.nii',
              r'D:/MRI/Nifti/634571/634571_100_raw.nii',]
              #r'D:/MRI/Nifti/634569/634569_100_raw.nii']
label_list = [r'D:/MRI/Nifti/636969/SkullStripAttempt1/SkullStripManual.nii',
              r'D:\MRI\Nifti\632298\SkullStrip\mask.nii',
              r'D:/MRI/Nifti/634571/SkullStrip/ManualSkullStrip.nii',]
              #r'D:/MRI/Nifti/634569/ManualSkullStrip.nii']

import itertools
order = np.array(list(itertools.permutations([0,1,2], 3)))



for ii,img_name in enumerate(image_list):
    Image = sitk.ReadImage(img_name)
    # Fix a bug in image loading
    Image.SetSpacing([.1,.1,.1])
    #Image = resample_img(Image,new_spacing=[.025,.025,.025],interpolator=sitk.sitkLinear)
    image = sitk.GetArrayFromImage(Image)
    #image = image.transpose((1,0,2))
    
    Label = sitk.ReadImage(label_list[ii])
    Label.SetSpacing([.1,.1,.1])
    #Label = resample_img(Label,new_spacing=[.025,.025,.025],interpolator=sitk.sitkNearestNeighbor)
    label = sitk.GetArrayFromImage(Label)>0
    #label = label.transpose((1,0,2))
    
    length, col, row = image.shape

    for i in range(0, length-patch_dims[0]+1, strides[0]):
        for j in range(0, col-patch_dims[1]+1, strides[1]):
            for k in range(0, row-patch_dims[2]+1, strides[2]):
                cur_patch=image[i:i+patch_dims[0],
                              j:j+patch_dims[1],
                              k:k+patch_dims[2]][:].reshape([1,
                                                             patch_dims[0],
                                                             patch_dims[1],
                                                             patch_dims[2]])
                cur_patch = np.transpose(cur_patch, (0, 2, 3, 1))
                patches.append(cur_patch)

                cur_label=label[i:i+patch_dims[0],
                                  j:j+patch_dims[1],
                                  k:k+patch_dims[2]][:].reshape([1,
                                                                 patch_dims[0],
                                                                 patch_dims[1],
                                                                 patch_dims[2]])
                cur_label = np.transpose(cur_label, (0, 2, 3, 1))
                labels.append(cur_label)

In [20]:
history  = model.fit(x = np.vstack(patches),y = np.vstack(labels),)

11/11 [==============================] - 114s 10s/step - loss: -0.6870 - dice_coef: 0.6908 - val_loss: -0.2223 - val_dice_coef: 0.2084


In [22]:
model.save(r'C:\Users\yoni.browning\OneDrive - Allen Institute\Documents\GitHub\RodentMRISkullStripping\yoniModelRetrain\retrain_different.h5')

In [23]:
%%time
keras_paras.model_path = r'C:\Users\yoni.browning\OneDrive - Allen Institute\Documents\GitHub\RodentMRISkullStripping\yoniModelRetrain\retrain_different.h5'

input_path =r'D:/MRI/Nifti/634569/634569_100_raw.nii'
output_path =  r'D:\MRI\Nifti\634569\TEST_634569_different.nii'
voxsize = .1
#brain_seg_prediction(input_path, output_path, voxsize, pre_paras, keras_paras,transpose = (1,0,2))


# load model
seg_net = load_model(keras_paras.model_path,
                     custom_objects={'dice_coef_loss': dice_coef_loss,
                                     'dice_coef': dice_coef})

imgobj = sitk.ReadImage(input_path)

# re-sample to given voxel size
resampled_imgobj = resample_img(imgobj,
                                new_spacing=[voxsize, voxsize, voxsize],
                                interpolator=sitk.sitkLinear)

img_array = sitk.GetArrayFromImage(resampled_imgobj)


normed_array = min_max_normalization(img_array)
out_label_map, out_likelihood_map = out_LabelHot_map_2D(normed_array,
                                                        seg_net,
                                                        pre_paras,
                                                        keras_paras)

out_label_img = sitk.GetImageFromArray(out_label_map.astype(np.uint8))
out_label_img.CopyInformation(resampled_imgobj)

resampled_label_map = resample_img(out_label_img,
                                   new_spacing=imgobj.GetSpacing(),
                                   new_size=imgobj.GetSize(),
                                   interpolator=sitk.sitkNearestNeighbor)
# Save the results
sitk.WriteImage(resampled_label_map, output_path)

Wall time: 1min 51s


In [19]:
model.fit?